In [1]:
import great_expectations as gx
import pandas as pd

# Sample DataFrame
data = {
    "id": [1, 2, 3, 4],
    "age": [25, 30, 31, None],
    "name": ["Rohan", "Amit", "Neha", "Sam"]
}
df = pd.DataFrame(data)

# Context
context = gx.get_context()

# Data Source (Pandas)
data_source = context.data_sources.add_pandas(name="inventory_parts")

# Add Data Asset **with the dataframe directly**
data_asset = data_source.add_dataframe_asset(
    name="inventory_parts_asset"
)
assert type(context).__name__ == "EphemeralDataContext"

# Create Batch Definition
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    "inventory_parts_batch"
)

# Pass dataframe here
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create Expectation Suite
suite = gx.ExpectationSuite(name="inventory_parts_suite")

suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column="name"))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeUnique(column="id"))
suite.add_expectation(gx.expectations.ExpectColumnToExist(column="age"))
suite.add_expectation(gx.expectations.ExpectColumnToExist(column="name"))
suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column="age"))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeBetween(
    column="age",
    min_value=18,
    max_value=60
))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeOfType(
    column="name",
    type_="object"
))
suite.add_expectation(gx.expectations.ExpectTableColumnsToMatchOrderedList(
    column_list=["id", "age", "name"]
))

# Add suite
context.suites.add(suite)

# Validate
validation_results = batch.validate(suite)

# Clean output formatting (fix: use expectation_config.type)
print("\n===== Validation Summary =====")
print("Success:", validation_results.success)
for r in validation_results.results:
    print(f"{r.expectation_config.type} => {'PASSED' if r.success else 'FAILED'}")


Calculating Metrics: 100%|██████████| 26/26 [00:00<00:00, 1078.40it/s]


===== Validation Summary =====
Success: False
expect_column_values_to_not_be_null => PASSED
expect_column_to_exist => PASSED
expect_column_values_to_be_of_type => PASSED
expect_column_values_to_be_unique => PASSED
expect_column_to_exist => PASSED
expect_column_values_to_not_be_null => FAILED
expect_column_values_to_be_between => PASSED
expect_table_columns_to_match_ordered_list => PASSED


In [2]:
context.add_data_docs_site(
    site_name="local_site",
    site_config={
        "class_name": "SiteBuilder",
        "store_backend": {"class_name": "TupleFilesystemStoreBackend", "base_directory": "uncommitted/data_docs/local_site"},
        "renderer": {"module_name": "great_expectations.render.renderer"},
        "view": {"module_name": "great_expectations.render.view"},
    }
)


InvalidKeyError: Data Docs Site `local_site` already exists in the Data Context.

In [3]:
print(context.list_data_docs_sites())


{'local_site': {'class_name': 'SiteBuilder', 'show_how_to_buttons': True, 'store_backend': {'class_name': 'TupleFilesystemStoreBackend', 'base_directory': 'C:\\Users\\rohan\\AppData\\Local\\Temp\\tmpgxegfy0e'}, 'site_index_builder': {'class_name': 'DefaultSiteIndexBuilder'}}}


In [5]:
context.build_data_docs()


{'local_site': 'file://C:\\Users\\rohan\\AppData\\Local\\Temp\\tmpgxegfy0e\\index.html'}

In [6]:
context.open_data_docs()
